In [1]:
import cv2
from pyzbar.pyzbar import decode
import ipywidgets.widgets as widgets
from IPython.display import display
from Functions.USBCamera import USBCamera
from threading import Thread
import time

class QRCodeProcessorTester:
    def __init__(self, camera: USBCamera, process_interval_ms=500):
        self.camera = camera
        self.process_interval_ms = process_interval_ms / 1000.0
        self.image_widget = widgets.Image(format='jpeg', width=camera.width // 2, height=camera.height // 2)
        self.label_widget = widgets.Label(value="None")
        display(widgets.VBox([self.image_widget, self.label_widget]))

        self.running = True
        self.last_process_time = 0
        self.thread = Thread(target=self._update, daemon=True)
        self.thread.start()

    def _update(self):
        while self.running:
            frame = self.camera.value
            if frame is not None:
                current_time = time.time()

                frame_with_overlay = frame

                if current_time - self.last_process_time >= self.process_interval_ms:
                    self.last_process_time = current_time

                    # Detekter QR-koder
                    qr_data = self.detect_qr_code(frame)
                    if qr_data:
                        self.label_widget.value = ", ".join(qr_data)
                    else:
                        self.label_widget.value = "None"

                # Oppdater widgeten
                frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                _, jpeg = cv2.imencode('.jpg', frame)
                self.image_widget.value = jpeg.tobytes()

    def detect_qr_code(self, frame):
        """Dekode QR-koder fra bildet."""
        decoded_objects = decode(frame)
        if decoded_objects:
            return [obj.data.decode('utf-8') for obj in decoded_objects]
        return []


    def stop(self):
        """Stopp oppdateringen."""
        self.running = False
        self.thread.join()

camera = USBCamera.instance(width=640, height=480)
qr_detector = QRCodeProcessorTester(camera, process_interval_ms=500)